# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

pprint(db.establishments.find_one())

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
results = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
num_docs = establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print('Number of Rows:', len(results))

# Display the first 10 rows of the DataFrame
result_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query =  {'RatingValue': {'$gte': '4'}}
fields = {'LocalAuthorityName': {'$regex': 'London'}}
new_q = dict(query, **fields)
rating_results = list(establishments.find(new_q))

# Use count_documents to display the number of documents in the result
london_est = establishments.count_documents(new_q)
print(f"London has {london_est} establishments with a Rating of four or more stars.")
print("----------------------------------------------------------------------------")
# Display the first document in the results using pprint
pprint(rating_results[0])


In [ ]:
# Convert the result to a Pandas DataFrame
london_result_df = pd.DataFrame(rating_results)
# Display the number of rows in the DataFrame
print(len(rating_results))
print("--")

# Display the first 10 rows of the DataFrame
london_result_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

# Rating value must equal 5
query = {'RatingValue': '5', 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
         }
# Sort by hygiene score
sort =  [('scores.Hygiene', 1)]

# Print the results
for results in establishments.find(query).sort(sort).limit(5):
    pprint(results)

In [ ]:
# Convert the result to a Pandas DataFrame
penang_result_df = pd.DataFrame(establishments.find(query).sort(sort).limit(5))

# Display the first 10 rows of the DataFrame
penang_result_df.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest


# Write an aggregation query that counts the number of documents
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': { '$sum': 1 }}}

# Create a match query 
match_query = {'$match': {'scores.Hygine': 0}}

# Create a dictionary that will allow the pipeline to sort  
# then count in descending order
sort_values = {'$sort': { 'count': -1 }}

# Put the pipeline together
pipeline = [group_query, match_query, sort_values]

results = list(establishments.aggregate(pipeline))


# Print the number of documents in the result
print(len(results))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
zero_hy_score_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(len(zero_hy_score_df))
# Display the first 10 rows of the DataFrame
zero_hy_score_df.head(10)